In [1]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import requests

# 마켓 정보 조회 API
url = "https://api.upbit.com/v1/market/all"
headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)
markets = response.json()

# BTC/KRW 마켓 필터링
btc_market = [market for market in markets if market['market'] == 'KRW-BTC'][0]
print(btc_market)


{'market': 'KRW-BTC', 'korean_name': '비트코인', 'english_name': 'Bitcoin'}


In [3]:
# 캔들 데이터 수집 함수
def fetch_upbit_data(market="KRW-BTC", count=200):
    url = "https://api.upbit.com/v1/candles/days"
    params = {
        "market": market,
        "count": count
    }
    response = requests.get(url, params=params)
    data = response.json()

    # 데이터프레임 변환
    df = pd.DataFrame(data)
    df = df[["candle_date_time_kst", "trade_price"]]
    df.columns = ["date", "price"]
    df["date"] = pd.to_datetime(df["date"])
    df.sort_values(by="date", inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# 데이터 가져오기
df = fetch_upbit_data()
print(df.head())


                 date       price
0 2024-05-25 09:00:00  96240000.0
1 2024-05-26 09:00:00  95610000.0
2 2024-05-27 09:00:00  96050000.0
3 2024-05-28 09:00:00  94600000.0
4 2024-05-29 09:00:00  93910000.0


In [4]:
# 정규화 및 시계열 데이터 생성
def preprocess_data(df, seq_length=30):
    # 가격 정규화
    scaler = MinMaxScaler(feature_range=(0, 1))
    df["price_normalized"] = scaler.fit_transform(df[["price"]])

    # 시계열 데이터 생성
    def create_sequences(data, seq_length):
        X, y = [], []
        for i in range(len(data) - seq_length):
            X.append(data[i:i + seq_length])
            y.append(data[i + seq_length])
        return np.array(X), np.array(y)

    X, y = create_sequences(df["price_normalized"].values, seq_length)

    # 데이터 분리
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test, scaler

# 데이터 전처리 실행
X_train, X_val, X_test, y_train, y_val, y_test, scaler = preprocess_data(df)
print("학습 데이터 크기:", X_train.shape, y_train.shape)


학습 데이터 크기: (136, 30) (136,)


In [5]:
# GRU 모델 생성
def build_gru_model(seq_length, input_dim, learning_rate):
    model = Sequential([
        GRU(128, return_sequences=True, input_shape=(seq_length, input_dim), kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        GRU(64, return_sequences=False, kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
        Dense(1)
    ])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["mae"])
    return model


In [6]:
# 학습률 스케줄
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9
)

# 조기 종료 설정
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)


In [7]:
print("X_train의 크기:", X_train.shape)

X_train의 크기: (136, 30)


In [8]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [9]:
print("변경 후 X_train의 크기:", X_train.shape)

변경 후 X_train의 크기: (136, 30, 1)


In [10]:
# 모델 생성
model = build_gru_model(seq_length=X_train.shape[1], input_dim=X_train.shape[2], learning_rate=lr_schedule)

# 학습
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 171ms/step - loss: 1.7993 - mae: 0.2782 - val_loss: 0.8353 - val_mae: 0.1244
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.7083 - mae: 0.1196 - val_loss: 0.3468 - val_mae: 0.0781
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.3066 - mae: 0.0737 - val_loss: 0.2117 - val_mae: 0.1115
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.1979 - mae: 0.0856 - val_loss: 0.1599 - val_mae: 0.0921
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.1484 - mae: 0.0717 - val_loss: 0.1088 - val_mae: 0.0494
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.1059 - mae: 0.0784 - val_loss: 0.0682 - val_mae: 0.0416
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0677 - mae: 0.0692 - val_loss: 0.0480 - val_mae: 0.0609
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0505 - mae: 0.0774 - val_loss: 0.0441 - val_mae: 0.0837
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0453 - mae:

In [11]:
# 테스트 데이터 평가
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"테스트 손실(MSE): {test_loss:.4f}, 테스트 평균 절대 오차(MAE): {test_mae:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0126 - mae: 0.0501
테스트 손실(MSE): 0.0126, 테스트 평균 절대 오차(MAE): 0.0501


In [12]:
# 미래 데이터 예측
def predict_future(model, X_test, scaler):
    last_sequence = X_test[-1].reshape(1, X_test.shape[1], X_test.shape[2])
    predicted_price = model.predict(last_sequence)
    predicted_price_original = scaler.inverse_transform(predicted_price)
    return predicted_price_original[0][0]

predicted_price = predict_future(model, X_test, scaler)
print(f"예측된 비트코인 가격: {predicted_price:.2f} KRW")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
예측된 비트코인 가격: 138348144.00 KRW
